In [2]:
# Install a pip package in the current Jupyter kernel
import sys
#!{sys.executable} -m pip install ipykernel
print(sys.executable)

/Users/liuzhixian/opt/anaconda3/bin/python


In [1]:
#-*- coding = utf-8 -*-



import numpy as np
import re   #正则表达，文字匹配
from bs4 import BeautifulSoup  #网页解析
import urllib.request, urllib.response , urllib.error#定制URL， 获取网页数据
import xlsxwriter as xlwt  #进行excel操作
import sqlite3  #进行SQLite数据库操作
import time #time counter
import wget #img download
import sys
import os #文件夹相关
import datetime #当前时间

In [55]:
def main():
    print("1: Yande.re img download. \n2: Study program for Douban")
    mode = input("Enter the function that you want(number): (Leave it empty to quit)")
    if mode == "":
        print("Exited!")
    elif mode == "1":
        yande_re_pic_saver()
    elif mode == "2":
        TestDouban()
    else:
        print("Unexpected function code!")


#爬取豆瓣信息的训练

def TestDouban():
    baseurl1 = "https://movie.douban.com/top250?start="
    getData(baseurl1)
    #保存路径
    #savepath= ".\\豆瓣电影top250.xls"
    #保存数据
    #saveData(savepath)

def getData(baseurl):
    datalist = []
    for i in range(0,10): #调用获取页面信息函数
        url = baseurl + str(i*25)
        html = askURL(url) #保存获取到的网页源码
        #解析数据
        soup = BeautifulSoup(html,"html.parser") #html.parser是解析器
        for item in soup.find_all('div', class_="item"): #查找符合要求的字符串，形成列表
            # print(item) #测试：查看电影item全部信息
            data = [] #保存一部电影的所有信息
            item = str(item) #转换成字符串
            #影片详情的链接
            link = re.findall(findLink,item)[0] #re库用来通过正则表达式查找指定的字符串
            data.append(link) #添加链接
            imgScr = re.findall(findImgScr,item)[0]
            data.append(imgScr)
            # print(re.findall(findImgScr,item)[0])

            titles = re.findall(findTitle,item)
            #片名可能只有一个中文名，没有外国名
            if(len(titles) == 2):
                ctitle = titles[0]
                data.append(ctitle)
                otitle = titles[1].replace("/","")
                otitle = otitle.replace(u'\xa0', u' ')
                data.append(otitle)
            else:
                data.append(titles[0])
                data.append(' ') #外国名留空

            rating = re.findall(findRating,item)[0] #评分
            data.append(rating)
            judgeNum = re.findall(findJudge,item)[0] #评价人数
            data.append(judgeNum)

            inq =  re.findall(findInq,item) #概况
            if len(inq)!=0:
                inq = inq[0].replace("。","")
                data.append(inq)
            else:
                data.append(" ")

            bd = re.findall(findBd,item)[0] #影片相关内容：导演，主演，年份，地区，类别
            bd = re.sub('<br(\s+)?/>(\s+)?'," ",bd) #去掉<br/>
            bd = re.sub('/'," ",bd) #替换/
            bd = bd.replace(u'\xa0', u' ')
            data.append(bd.strip()) #去掉前后的空格
            datalist.append(data) #把处理好的一部电影信息放入datalist

    print(datalist)
    return datalist

 #保存数据
def saveData(savepath):
    print("not yet")
#得到指定一个URL网页信息
#影片详情链接的规则
findLink = re.compile(r'<a href="(.*?)">') #创建正则表达式对象，表示规则（字符串的模式） (.*?) .*这个概念有0个或者1个.
#影片图片
findImgScr = re.compile(r'<img alt=".*?" class=".*?" src="(.*)" width="100"/>',re.S) #re.S让 换行符 包含在字符中  . 不包含换行符
#影片片名
findTitle = re.compile(r'<span class="title">(.*)</span>')
#影片评分
findRating = re.compile(r'<span class="rating_num" property="v:average">(.*)</span>')
#评价人数
findJudge = re.compile(r'<span>(\d*)人评价</span>')
#概况
findInq = re.compile(r'<span class="inq">(.*)</span>')
#影片相关内容：导演，主演，年份，地区，类别
findBd = re.compile(r'<p class="">(.*?)</p>',re.S)

#爬取Yande.re网页
def yande_re_pic_saver():
    #Yande.re 图片下载
    baseurl2seg1 = "https://yande.re/post?page="
    SearchingTags = input("Enter a valid searching tags(example: fuzichoko, enter 'popular to download today's popular')")
    baseurl2seg2 = "&tags="

    #下载当日popular
    if SearchingTags == "popular":
        period = "d"
        period = str(input("Input wanted popular period:('d','w','m','y', 24h when leave it empty)"))
        popurl = "https://yande.re/post/popular_recent?period=" + "1"+ period
        path = "./"
        path = str(input("Please enter a valid save path.(You can keep path empty,there is a default path.1 is for test)"))
        if path == "1":
            path = "/Users/liuzhixian/Desktop/"
        #当前时间
        date = str(datetime.date.today())
        #判断是否创建新文件夹
        if os.path.exists(path+ str(SearchingTags)+"_"+period+"_"+date) == 0: 
            os.makedirs(path+ str(SearchingTags)+"_"+period+"_"+date)
            print("Folder "+str(SearchingTags)+"_"+period+"_"+date+" created successfully!")
        datalist = [] #reset datalist
        #爬取网页
        time_start = time.time() #统计爬取网页时间
        html = askURL(popurl)
        #逐一解析网页
        bs = BeautifulSoup(html,"html.parser")
        t_list = bs.select("a[class='directlink largeimg']")
        #分割单一图片URL
        for link in t_list:
            datalist.append(link.get('href'))
        #保存数据
        for k in range(0,len(datalist)):
            imgsavepath = path+ str(SearchingTags)+"_"+period+"_"+date + "/"+ str(k)+".jpg"
            wget.download(datalist[k],imgsavepath)
        time_end = time.time()
        time_spent = (time_end - time_start)
        #输出爬取进度
        print("Download popular img success! Time spent = %.2fs"%(time_spent))
        

    else:
        while BeautifulSoup(askURL(baseurl2seg1 + baseurl2seg2 + SearchingTags),"html.parser").select("a[class='directlink largeimg']") == []:#测试Tag是否存在
            if  SearchingTags == "magicexitcode":    
                sys.exit("Successfully exit!")
            print("Tag is invalid, please enter a valid tag.")
            SearchingTags = input("Re-enter a valid searching tags(example: fuzichoko,enter 'magicexitcode' to exit)")
        m = int(input("Please enter the start page number(positive number>=1): "))
        n = int(input("Please enter the end page number(positive number): "))
        #输入保存地址
        path = "./"
        path = str(input("Please enter a valid save path.(You can keep path empty,there is a default path.1 is for test)"))+"/"
        if path == "1/":
            path = "/Users/liuzhixian/Desktop/"
        #判断是否创建新文件夹
        if os.path.exists(path+ str(SearchingTags)) == 0: 
            os.makedirs(path+ str(SearchingTags))
            print("Folder "+str(SearchingTags)+" created successfully!")
        counter = 0
        for i in range(m,n+1): #调用获取页面信息函数
            datalist = [] #reset datalist
            #爬取网页
            time_start = time.time() #统计爬取网页时间
            url  = baseurl2seg1 + str(i) + baseurl2seg2 + str(SearchingTags)
            html = askURL(url)
            #逐一解析网页
            bs = BeautifulSoup(html,"html.parser")
            t_list = bs.select("a[class='directlink largeimg']")
            #判断是否爬到了最后一页
            if t_list == []:
                print("Spider has reached the final page! Total page = %d"%(i-1)) 
                break
            #分割单一图片URL
            for link in t_list:
                datalist.append(link.get('href'))
            #保存数据
            for k in range(0,len(datalist)):
                imgsavepath = path+ str(SearchingTags) + "/"+ str(k+counter)+".jpg"
                wget.download(datalist[k],imgsavepath)
            counter = counter + len(datalist)
            time_end = time.time()
            time_spent = (time_end - time_start)
            #输出爬取进度
            print("Page%d success! Time spent = %.2fs"%(i,time_spent))
        
def askURL(url):
    #设置用户代理（告诉浏览器我们可以接受什么类型的文件）， 模拟header向服务器发送消息
    head = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.3 Safari/605.1.15"
    }
    req = urllib.request.Request(url, headers=head)
    html = ""
    try:
        response = urllib.request.urlopen(req,timeout= 10)
        html = response.read().decode("utf-8")
    except urllib.error.URLError as e:
        if hasattr(e,"code"):
            print(e.code)
        if hasattr(e,"reason"):
            print(e.reason)
    return html

In [54]:

#Start
if __name__ == "__main__" :  #当主程序执行时
    main()

1: Yande.re img download. 
2: Study program for Douban
[['https://movie.douban.com/subject/1292052/', 'https://img2.doubanio.com/view/photo/s_ratio_poster/public/p480747492.jpg', '肖申克的救赎', '  The Shawshank Redemption', '9.7', '2829701', '希望让人自由', '导演: 弗兰克·德拉邦特 Frank Darabont   主演: 蒂姆·罗宾斯 Tim Robbins  ... 1994   美国   犯罪 剧情'], ['https://movie.douban.com/subject/1291546/', 'https://img1.doubanio.com/view/photo/s_ratio_poster/public/p2561716440.jpg', '霸王别姬', ' ', '9.6', '2097056', '风华绝代', '导演: 陈凯歌 Kaige Chen   主演: 张国荣 Leslie Cheung   张丰毅 Fengyi Zha... 1993   中国大陆 中国香港   剧情 爱情 同性'], ['https://movie.douban.com/subject/1292720/', 'https://img2.doubanio.com/view/photo/s_ratio_poster/public/p2372307693.jpg', '阿甘正传', '  Forrest Gump', '9.5', '2118746', '一部美国近现代史', '导演: 罗伯特·泽米吉斯 Robert Zemeckis   主演: 汤姆·汉克斯 Tom Hanks   ... 1994   美国   剧情 爱情'], ['https://movie.douban.com/subject/1292722/', 'https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2889314814.jpg', '泰坦尼克号', '  Titanic', '9.5', '2

In [6]:
#urllib test
import urllib.request
import urllib.parse
import urllib.error
#get request
#response = urllib.request.urlopen("https://yande.re/post?tags=yandere")
#print(response.read().decode('utf-8')) #对获取的网页源码进行utf-8解码

#post request 模拟用户真实登陆，包括密码，cookie
#urltest = "https://httpbin.org/post"
#data = bytes(urllib.parse.urlencode({"hello": "world"}),encoding="utf-8")
#response = urllib.request.urlopen(urltest,data= data)
#print(response.read().decode('utf-8'))

#timeout
#try:
#    urltest = "https://httpbin.org/get"
#    response = urllib.request.urlopen(urltest,timeout= 0.01)
#    print(response.read().decode('utf-8'))
#except urllib.error.URLError as e: 
#    print("time out!")

#getheader
#urltest = "https://www.baidu.com"
#response = urllib.request.urlopen(urltest)
#print(response.status)
#print(response.getheaders())
#print(response.getheader("Date"))

#伪装成浏览器
urltest1 = "https://httpbin.org/post"
urltest2 = "http://www.douban.com"
headers = {
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.3 Safari/605.1.15"
}
#data = bytes(urllib.parse.urlencode({'name':'testor'}),encoding="utf-8")
#req = urllib.request.Request(url=urltest1,data= data,headers= headers,method= "POST")
#response = urllib.request.urlopen(req)
#print(response.read().decode('utf-8'))

req = urllib.request.Request(url=urltest2,headers= headers,method= "GET")
response = urllib.request.urlopen(req)
print(response.read().decode('utf-8'))




<!DOCTYPE HTML>
<html lang="zh-cmn-Hans" class="ua-mac ua-webkit">
<head>
<meta charset="UTF-8">
<meta name="google-site-verification" content="ok0wCgT20tBBgo9_zat2iAcimtN4Ftf5ccsh092Xeyw" />
<meta name="description" content="提供图书、电影、音乐唱片的推荐、评论和价格比较，以及城市独特的文化生活。">
<meta name="keywords" content="豆瓣,小组,电影,同城,豆品,广播,登录豆瓣">
<meta property="qc:admins" content="2554215131764752166375" />
<meta property="wb:webmaster" content="375d4a17a4fa24c2" />
<meta name="mobile-agent" content="format=html5; url=https://m.douban.com">
<title>豆瓣</title>
<script>
function set_cookie(t,e,o,n){var i,a,r=new Date;r.setTime(r.getTime()+24*(e||30)*60*60*1e3),i="; expires="+r.toGMTString();for(a in t)document.cookie=a+"="+t[a]+i+"; domain="+(o||"douban.com")+"; path="+(n||"/")}function get_cookie(t){var e,o,n=t+"=",i=document.cookie.split(";");for(e=0;e<i.length;e++){for(o=i[e];" "==o.charAt(0);)o=o.substring(1,o.length);if(0===o.indexOf(n))return o.substring(n.length,o.length).replace(/\"/g,"")}return null}win

BeautifulSoup4 将复杂HTML文档转换成一个复杂的树形结构，每个节点都是python对象，所有对象都可以归纳为四种：
- Tag
- NavigableString
- BeautifulSoup
- Comment

In [8]:
# Test for beautifulsoup
from bs4 import BeautifulSoup
file = open("./Test.html","rb")
html = file.read()

# 用html.parser解析
bs = BeautifulSoup(html,"html.parser")
import re
#print(bs.title)
#print(bs.a)
#print(bs.head)
#print(type(bs.head))
#1.Tag 标签及其内容；拿到它所找的到第一个标签

#print(bs.title.string)
#print(type(bs.title.string))
#2.NavigableString，标签里的内容（字符串）

#print(bs.a.attrs)
#拿标签里的属性

#print(type(bs))
#print(bs.name)
#print(bs.attrs)
#print(bs)
#3.BeautifulSoup 表示整个文档

#print(bs.a.string)
#4.comment 会把注释符号去除

#---------------------------------------------------------

#节点获取，文档的遍历
#print(bs.head.contents)
#i = 7
#print("The %dth: "%i + str(bs.head.contents[i]))
#更多内容见 https://www.crummy.com/software/BeautifulSoup/bs4/doc.zh/#id19

#文档的搜索
#find_all 查找所有与字符串匹配的内容
#t_list = bs.find_all("a")
#print(t_list)

#正则表达式搜索：search()
#t_list = bs.find_all(re.compile("a"))
#print(t_list)

#传人一个函数，根据函数的要求来搜索
#def name_exists(tag):
#    return tag.has_attr("name")
#t_list = bs.find_all(name_exists)
#print(t_list)

#keywords 参数
#t_list = bs.find_all(id = "head")
#t_list = bs.find_all(class_ = True) #所有拥有class
#t_list = bs.find_all(class_ = "mnav")

#text 参数
#t_list = bs.find_all(text = "hao123")
#t_list = bs.find_all(text = ["hao123","新闻","更多产品"])
#t_list = bs.find_all(text = re.compile("\d"))

#limit 参数
#t_list = bs.find_all("a",limit=3)
#for item in t_list:
#    print(item)

#css 选择器
#t_list = bs.select('title') #通过标签查找
t_list = bs.select(".mnav")  #通过类名查找
#t_list = bs.select("#u1") #通过id查找
#t_list = bs.select("a[class='bri']") #通过属性来查找
#t_list = bs.select("head>title") #div>div>div... 通过子标签来查找
#t_list = bs.select(".mnav ~ .bri") #通过兄弟标签查找

#print(t_list[0].get_text())

for item in t_list:
    print(item)

<a class="mnav" href="http://news.baidu.com" name="tj_trnews"><!--新闻注释--></a>
<a class="mnav" href="http://news.baidu.com" name="tj_trnews">新闻</a>
<a class="mnav" href="http://www.hao123.com" name="tj_trhao123">hao123</a>


In [9]:
#正则表达式
#更多可见 https://www.runoob.com/regexp/regexp-syntax.html, https://blog.csdn.net/zyc88888/article/details/98479629


In [10]:
#error test
#response = urllib.request.urlopen("https://yande.re/post?page=14&tags=fuzichoko")
#bs = BeautifulSoup(response,"html.parser")
#result = bs.select("a[class='directlink largeimg']")
#for link in result:
#    print(link.get('href'))


In [56]:
#test xlwt

workbook = xlwt.Workbook(encoding="utf-8") #创建workbook对象
worksheet = workbook.add_sheet("sheet1") #创建工作表
worksheet.write(0,0, 'hello') #写入数据，参数分别是行、列、数据
workbook.save("test1.xlsx") #保存数据表

TypeError: __init__() got an unexpected keyword argument 'encoding'